# Classification on Splice-junction Gene Sequences 
This file takes the implemenation for hyperparameter search. 
Codes for hparam search and for tensorboard make it long.


In [1]:
#Imports
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

MY_MODE_TRAINING = True
MY_MODE_PREDICTION = False
LOGDIR = './TMP/'

### Load the dataset

In [2]:
def load_data(file_path) :
    df = pd.read_csv(file_path, header=None)
    df.columns = ['classlabel', 'name', 'sequence']
    df.tail()
    
    return df

### Pre-process the dataset
* Apply one-hot-encoding to input data
* Take 20% as test data 

In [3]:
def preprocess_data(df) :
    
    # Encoding class labels
    class_le = LabelEncoder()
    y = class_le.fit_transform(df['classlabel'].values)
    #print("y:",y)
    
    # Encoding sequence
    # Here we use one hot encoding to encode the character in DNA sequence. 
    # So each dna sequence is converted to a 60x8 2D array 
    def Seq2Vec(seq):
        s = str(seq).strip()
        CharDict = { "A":[0,0,0,0,0,0,0,1],
                     "G":[0,0,0,0,0,0,1,0],
                     "C":[0,0,0,0,0,1,0,0],
                     "T":[0,0,0,0,1,0,0,0],
                     "D":[0,0,0,1,0,0,0,0],
                     "N":[0,0,1,0,0,0,0,0],
                     "S":[0,1,0,0,0,0,0,0],
                     "R":[1,0,0,0,0,0,0,0]}
        return np.asarray([CharDict[c] for c in s], dtype=np.float32).flatten()

    df['seqvec'] = df['sequence'].apply(Seq2Vec)
    X = np.vstack(df['seqvec'].values)
    print("Total samples:", X.shape[0])
    
    # Split the data set into training/test set
    sss = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=0)
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

    print("Training samples: ", X_train.shape[0], "Test samples: ", X_test.shape[0])
    
    return X_train, y_train, X_test, y_test

### DNN model

In [4]:
def model_dnn(X_train, y_train, X_test, y_test, hparam, batch_size=100, n_epochs=2000, 
              use_two_conv=True, n_fc=3, mode_learning=0) :
    
    #DNN approach
    '''
    feature_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
    dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units=[300, 100], n_classes=3,
                                             feature_columns=feature_columns)
    dnn_clf.fit(x=X_train, y=y_train, batch_size=50, steps=n_epochs)

    from sklearn.metrics import accuracy_score

    y_pred = list(dnn_clf.predict(X_test))
    accuracy = accuracy_score(y_test, y_pred)
    accuracy
    '''
    
    tf.reset_default_graph()
 
    # Hyper parameters
    assert( n_fc>=1 & n_fc<=4 )
    if( n_fc==4 ) : 
        fc1_node = 300
        fc2_node = 100
        fc3_node = 30
    elif( n_fc==3 ) : 
        fc2_node = 300
        fc3_node = 100
    elif( n_fc==2 ) :
        fc3_node = 200
        
    beta = 0.01                  # Regularization 0.01
    dropout_rate = 0.5           # Dropout rate for dropout layer
    learning_rate_set = [0.001, 1e-3, 1e-4, 1e-5]
    assert( mode_learning>=0 & mode_learning<=3 )
    starter_learning_rate = learning_rate_set[mode_learning]
    
    # Variables and inputs
    X = tf.placeholder(tf.float32, shape=(None, 480), name="X")
    input = tf.reshape(X, [-1, 60*8])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    mode = tf.placeholder(tf.bool, shape=(None), name="mode")
    scale = tf.placeholder(tf.float32, shape=(None), name="scale")
    global_step = tf.placeholder(tf.int32, shape=(None), name="global_step")
    learning_rate = tf.Variable(starter_learning_rate, trainable=False)
    
    if( n_fc == 4 ) :
        # Add the 1st fully connected layer, apply L2 regularizer and xavier weight initializer
        fc1 = tf.layers.dense(
            inputs=input, 
            units=fc1_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="fc1")

        #Adding a dropout layer to avoid overfitting
        dropout1 = tf.layers.dropout(inputs=fc1, rate=dropout_rate, training=mode )
     
    if( n_fc>=3 ) : 
        # Add the 2nd fully connected layer
        fc2 = tf.layers.dense(
            inputs= input if n_fc==3 else dropout1, 
            units=fc2_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="fc2")
    
        #Adding a dropout layer to avoid overfitting
        dropout2 = tf.layers.dropout(inputs=fc2, rate=dropout_rate, training=mode )
    
    if( n_fc>=2 ) : 
        # Add the 2nd fully connected layer
        fc3 = tf.layers.dense(
            inputs= input if n_fc==2 else dropout2, 
            units=fc3_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="fc3")

        #Adding a dropout layer to avoid overfitting
        dropout3 = tf.layers.dropout(inputs=fc3, rate=dropout_rate, training=mode )
    
    # The last output layer
    logits = tf.layers.dense(
        inputs=input if n_fc==1 else dropout3, 
        units=3,
        activation=tf.nn.softmax,
        kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
        kernel_initializer=tf.contrib.layers.xavier_initializer(),
        name="output_FC")
    
    # Cross entropy
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    # Apply regularization in training stage and not in prediction stage
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(entropy) + beta*sum( tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) )
    tf.summary.scalar('loss', loss)

    with tf.name_scope('train'):
        if(mode_learning==0):
            # Decayed learning rate
            # Start learning rate as 0.001, decay rate as 0.5, decay 6 steps to ~0.000015 (~1e-5)
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                       decay_steps=n_epochs//6, decay_rate=0.5, staircase=True)
        tf.summary.scalar('learning_rate', learning_rate)
        # Use Adam optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()

    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(hparam + 'train', sess.graph)
        test_writer = tf.summary.FileWriter(hparam + "test")
        
        init.run()
        n_rounds = X_train.shape[0] // batch_size
        for epoch in range(n_epochs):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train, epoch, n_rounds)
            summary_train, _ = sess.run([merged, optimizer], 
                                        feed_dict= {X: X_batch, y: y_batch, 
                                                    mode: MY_MODE_TRAINING, scale: beta,
                                                    global_step: epoch}) 
            train_writer.add_summary(summary_train, epoch)

            if (epoch % 10 == 0):
                _, acc_train = sess.run([merged, accuracy], feed_dict={X: X_train, y: y_train, 
                                                    mode: MY_MODE_PREDICTION, scale: 0,
                                                    global_step: epoch})
                #train_writer.add_summary(summary_train, epoch)
                
                summary_test, acc_test = sess.run([merged, accuracy], 
                                                  feed_dict={X: X_test, y:y_test, 
                                                            mode: MY_MODE_PREDICTION, scale: 0,
                                                            global_step: epoch})
                test_writer.add_summary(summary_test, epoch)
               
                print(epoch, "Train accuracy:", acc_train,  "Test_accuracy:", acc_test)
    

### Next batch - support function
* Get the next mini_batch from training data

In [5]:
def next_batch(num, data, labels, epoch, rounds):
    ''' 
    Return a total of `num` random samples and labels. 
    Reshuffle the index when running over the total data set epoch%rounds==0
    
    Arg: 
      num, the number of returned data size
      data, the input X
      labels, the label y
      epoch, the current epoch value
      rounds, rounds = label_size // num 

    Returns:
      Return "num" of X and y array
    '''
    global g_idx
    set_cnt = epoch % rounds
    if( (set_cnt) == 0 ) :
        #print("Reshuffling...")
        g_idx = np.arange(0, labels.shape[0])
        np.random.shuffle(g_idx)
        

    idx = g_idx[set_cnt*num:set_cnt*num+num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

### CNN Model

In [6]:
def model_cnn(X_train, y_train, X_test, y_test, hparam, batch_size=100, n_epochs=2000, 
              use_two_conv=True, n_fc=3, mode_learning=0) :
    
    #CNN approach
    
    tf.reset_default_graph()
    
    #Hyper parameters
    conv1_depth = 32
    conv2_depth = 64
    conv1_kernel_size = [3, 3]
    conv2_kernel_size = [3, 3]
    dense1_node = 64
    dense2_node = 16
    beta = 0.01                  # Regularization 0.01
    dropout_rate = 0.5           # Dropout rate for dropout layer
    learning_rate_set = [0.01, 1e-3, 1e-4, 1e-5]
    assert( mode_learning>=0 & mode_learning<=3 )
    starter_learning_rate = learning_rate_set[mode_learning]

    X = tf.placeholder(tf.float32, shape=(None, 480), name="X")
    input = tf.reshape(X, [-1, 60, 8, 1])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    mode = tf.placeholder(tf.bool, shape=(None), name="mode")
    scale = tf.placeholder(tf.float32, shape=(None), name="scale")
    global_step = tf.placeholder(tf.int32, shape=(None), name="global_step")
    learning_rate = tf.Variable(starter_learning_rate, trainable=False)
    
    if( use_two_conv ) :   # Use two conv layer
        # Convolutional Layer #1
        # Computes 32 features using a 3x3 filter.
        # Padding is added to preserve width and height.
        # Input Tensor Shape: [batch_size, 60, 8, 1]
        # Output Tensor Shape: [batch_size, 60, 8, 32]
        conv1 = tf.layers.conv2d(
            inputs=input,
            filters=conv1_depth,
            kernel_size=conv1_kernel_size,
            padding="same",
            activation=tf.nn.relu,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
            name="conv1")

        #Adding a pooling layer
        pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

        # Convolutional Layer #2
        # Computes 64 features using a 3x3 filter.
        # Padding is added to preserve width and height.
        # Input Tensor Shape: [batch_size, 30, 4, 32]
        # Output Tensor Shape: [batch_size, 30, 4, 64]
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=conv2_depth,
            kernel_size=conv2_kernel_size,
            padding="same",
            activation=tf.nn.relu,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
            name="conv2")

        #Adding a pooling layer
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
        conv_out_flat = tf.reshape(pool2, [-1, 15 * 2 * conv2_depth])
        
    else :   # Use one conv layer
        conv2 = tf.layers.conv2d(
            inputs=input,
            filters=conv2_depth,
            kernel_size=conv2_kernel_size,
            padding="same",
            activation=tf.nn.relu,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(),
            name="conv")

        #Adding a pooling layer
        pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
        conv_out_flat = tf.reshape(pool2, [-1, 30 * 4 * conv2_depth])
        
    if( n_fc==3 ) : # Use 3 FC layer
        #Adding a fully connected layer
        dense1 = tf.layers.dense(
            inputs=conv_out_flat, 
            units=dense1_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="dense1")

        #Adding a dropout layer to avoid overfitting
        dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate, training=mode )  

        # Add the 2nd fully connected layer
        dense2 = tf.layers.dense(
            inputs=dropout1, 
            units=dense2_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="dense2")

        # The last output layer
        logits = tf.layers.dense(
            inputs=dense2, 
            units=3,
            activation=tf.nn.softmax,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="output_FC")
        
    elif( n_fc==2 ) : # Use 2 FC layer
        #Adding a fully connected layer
        dense1 = tf.layers.dense(
            inputs=conv_out_flat, 
            units=dense1_node, 
            activation=tf.nn.relu, 
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="dense1")

        #Adding a dropout layer to avoid overfitting
        dropout1 = tf.layers.dropout(inputs=dense1, rate=dropout_rate, training=mode )  

        # The last output layer
        logits = tf.layers.dense(
            inputs=dropout1, 
            units=3,
            activation=tf.nn.softmax,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="output_FC")
        
    else : # Use 1 FC layer
        # The last output layer
        logits = tf.layers.dense(
            inputs=conv_out_flat, 
            units=3,
            activation=tf.nn.softmax,
            kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=scale, scope=None),
            kernel_initializer=tf.contrib.layers.xavier_initializer(),
            name="output_FC")
        
    # Cross entropy
    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    # Apply regularization in training stage and not in prediction stage
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(entropy) + beta*sum( tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) )
    tf.summary.scalar('loss', loss)

    with tf.name_scope('train'):
        if(mode_learning==0):
            # Decayed learning rate
            # Start learning rate as 0.01, decay rate as 0.5, decay 10 steps to ~0.0000097 (~1e-5)
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                       decay_steps=n_epochs//10, decay_rate=0.5, staircase=True)
        tf.summary.scalar('learning_rate', learning_rate)
        # Use Adam optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()


    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(hparam + 'train', sess.graph)
        test_writer = tf.summary.FileWriter(hparam + "test")
        
        init.run()
        n_rounds = X_train.shape[0] // batch_size
        for epoch in range(n_epochs):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train, epoch, n_rounds)
            summary_train, _ = sess.run([merged, optimizer], 
                                        feed_dict= {X: X_batch, y: y_batch, 
                                                    mode: MY_MODE_TRAINING, scale: beta,
                                                    global_step: epoch}) 
            train_writer.add_summary(summary_train, epoch)

            if (epoch % 10 == 0):
                _, acc_train = sess.run([merged, accuracy], 
                                        feed_dict={X: X_train, y: y_train, 
                                                   mode: MY_MODE_PREDICTION, scale: 0,
                                                   global_step: epoch})
                #train_writer.add_summary(summary_train, epoch)
                
                summary_test, acc_test = sess.run([merged, accuracy], 
                                                  feed_dict={X: X_test, y:y_test, 
                                                            mode: MY_MODE_PREDICTION, scale: 0,
                                                            global_step: epoch})
                test_writer.add_summary(summary_test, epoch)
               
                print(epoch, "Train accuracy:", acc_train,  "Test_accuracy:", acc_test)
                

### RNN model

In [7]:
def model_rnn(X_train, y_train, X_test, y_test, hparam, batch_size=100, n_epochs=2000, 
              n_node=80, n_fc_node=64, mode_learning=0) :
    
    #RNN approach

    tf.reset_default_graph()

    # Input/output data dimension
    n_steps =  60
    n_inputs = 8
    n_outputs = 3
    
    #Hyper parameters
    n_neurons = n_node
    fc1_node = n_fc_node
    beta = 0.2                   # Regularization 0.2
    dropout_rate = 0.5           # Dropout layer for regularization
    learning_rate_set = [0.001, 1e-3, 1e-4, 1e-5]
    assert( mode_learning>=0 & mode_learning<=3 )
    starter_learning_rate = learning_rate_set[mode_learning]

    X = tf.placeholder(tf.float32, shape=(None, n_steps * n_inputs), name="X")
    input = tf.reshape(X, [-1, n_steps, n_inputs])
    y = tf.placeholder(tf.int32, shape=(None), name="y")
    mode = tf.placeholder(tf.bool, shape=(None), name="mode")
    scale = tf.placeholder(tf.float32, shape=(None), name="scale")
    global_step = tf.placeholder(tf.int32, shape=(None), name="global_step")
    learning_rate = tf.Variable(starter_learning_rate, trainable=False)
    
    # RNN layer
    basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
    outputs, states = tf.nn.dynamic_rnn(basic_cell, input, dtype=tf.float32)
        
    # Dropout layer to avoid overfitting
    dropout = tf.layers.dropout(inputs=states, rate=dropout_rate, training= mode )

    if( n_fc_node>0 ) : # Try out two FC layers if fc_node is non-zero
        # Fully connected layer - Xavier initializer and L2 regularizer
        fc1 = tf.contrib.layers.fully_connected(
            inputs=dropout, 
            num_outputs=fc1_node, 
            weights_initializer=tf.contrib.layers.xavier_initializer(),
            weights_regularizer=tf.contrib.layers.l2_regularizer(scale=beta, scope=None),
            activation_fn=tf.nn.relu 
        )
        # Dropout layer to avoid overfitting
        dropout2 = tf.layers.dropout(inputs=fc1, rate=dropout_rate, training= mode )

    # Fully connected layer - Xavier initializer and L2 regularizer
    logits = tf.contrib.layers.fully_connected(
        inputs=dropout if n_fc_node==0 else dropout2,
        num_outputs=n_outputs, 
        weights_initializer=tf.contrib.layers.xavier_initializer(),
        weights_regularizer=tf.contrib.layers.l2_regularizer(scale=beta, scope=None),
        #activation_fn=tf.nn.softmax 
    )

    entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    
    # Apply regularization in training stage and not in prediction stage
    with tf.name_scope('loss'):
        loss = tf.reduce_mean(entropy) + beta*sum( tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES) )
    tf.summary.scalar('loss', loss)

    with tf.name_scope('train'):
        if(mode_learning==0):
            # Decayed learning rate
            # Start learning rate as 0.001, decay rate as 0.5, decay 6 steps to ~0.000015 (~1e-5)
            learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                                       decay_steps=n_epochs//6, decay_rate=0.5, staircase=True)
        tf.summary.scalar('learning_rate', learning_rate)
        # Use Adam optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    
    
    init = tf.global_variables_initializer()

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(hparam + 'train', sess.graph)
        test_writer = tf.summary.FileWriter(hparam + "test")
        
        init.run()
        n_rounds = X_train.shape[0] // batch_size
        for epoch in range(n_epochs):
            X_batch, y_batch = next_batch(batch_size, X_train, y_train, epoch, n_rounds)
            summary_train, _ = sess.run([merged, optimizer], 
                                        feed_dict= {X: X_batch, y: y_batch, 
                                                    mode: MY_MODE_TRAINING, scale: beta,
                                                    global_step: epoch}) 
            train_writer.add_summary(summary_train, epoch)

            if (epoch % 10 == 0):
                _, acc_train = sess.run([merged, accuracy], 
                                        feed_dict={X: X_train, y: y_train, 
                                                   mode: MY_MODE_PREDICTION, scale: 0,
                                                   global_step: epoch})
                #train_writer.add_summary(summary_train, epoch)
                
                summary_test, acc_test = sess.run([merged, accuracy], 
                                                  feed_dict={X: X_test, y:y_test, 
                                                            mode: MY_MODE_PREDICTION, scale: 0,
                                                            global_step: epoch})
                test_writer.add_summary(summary_test, epoch)
               
                print(epoch, "Train accuracy:", acc_train,  "Test_accuracy:", acc_test)
            



### Main 

In [8]:
def make_hparam_string(model_name, mode_learning, n_fc, n_conv):
    conv_param = "conv=%d" % (n_conv)
    return LOGDIR+model_name+",lr_%d,%s,fc=%d," % (mode_learning, conv_param, n_fc)

def main(unused_argv):
    
    # Load data
    df = load_data("..\data\splice.data")
    
    # Preprocess data and split training and validation data
    X_train, y_train, X_test, y_test = preprocess_data(df)
    
    # DNN model
    '''
    for mode_learning in [0, 1, 2, 3]:
        for n_fc in [4, 3, 2, 1]:
            hparam = make_hparam_string("dnn", mode_learning=mode_learning, n_fc=n_fc, n_conv=0)
            print('Starting run for %s' % hparam)
            
            model_dnn(X_train, y_train, X_test, y_test, hparam=hparam, batch_size=100, n_epochs=5000, 
                      n_fc=n_fc, mode_learning=mode_learning)
    '''
        
    # CNN model
    '''
    # Search hyper parameters over the different # of layers
    for n_fc in [3, 2, 1]:
        for n_conv in [2, 1]:
            # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
            hparam = make_hparam_string("cnn_1", 2, n_fc, n_conv=n_conv)

            print('Starting run for %s' % hparam)
            model_cnn(X_train, y_train, X_test, y_test, batch_size=100, n_epochs=3000, hparam=hparam,
                      use_two_conv=(n_conv==2), n_fc=n_fc, mode_learning=2)
    '''
    '''
    # Search and compare the best learning rate
    for mode_learning in [0, 1, 2, 3]:
        n_fc=3
        for two_conv in [1]:
            # Construct a hyperparameter string for each one (example: "lr_1E-3,fc=2,conv=2)
            hparam = make_hparam_string("cnn_2", mode_learning=mode_learning, n_fc=n_fc, n_conv=n_conv)

            print('Starting run for %s' % hparam)
            model_cnn(X_train, y_train, X_test, y_test, batch_size=100, n_epochs=3000, hparam=hparam,
                      use_two_conv=(n_conv==2), n_fc=n_fc, mode_learning=mode_learning)
    '''
    
    # RNN model
    #model_rnn(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, batch_size=100, n_epochs=3000)
    for mode_learning in [1]:
        for n_node in [32, 64, 80, 150]:
            for n_fc_node in [0]:
                hparam = LOGDIR+"rnn,lr_%d,rnn_node=%d,fc_node=%d," % (mode_learning, n_node, n_fc_node)
                print('Starting run for %s' % hparam)

                model_rnn(X_train, y_train, X_test, y_test, hparam=hparam, batch_size=100, n_epochs=3000, 
                          n_node=n_node, n_fc_node=n_fc_node, mode_learning=mode_learning)  


In [9]:
if __name__ == "__main__":
  tf.app.run()

Total samples: 3190
Training samples:  2552 Test samples:  638
Starting run for ./TMP/rnn,lr_1,rnn_node=32,fc_node=0,
0 Train accuracy: 0.402821 Test_accuracy: 0.376176
10 Train accuracy: 0.622257 Test_accuracy: 0.598746
20 Train accuracy: 0.695141 Test_accuracy: 0.675549
30 Train accuracy: 0.548589 Test_accuracy: 0.545455
40 Train accuracy: 0.518809 Test_accuracy: 0.518809
50 Train accuracy: 0.520768 Test_accuracy: 0.520376
60 Train accuracy: 0.538009 Test_accuracy: 0.539185
70 Train accuracy: 0.520768 Test_accuracy: 0.520376
80 Train accuracy: 0.586991 Test_accuracy: 0.581505
90 Train accuracy: 0.545455 Test_accuracy: 0.53605
100 Train accuracy: 0.538401 Test_accuracy: 0.528213
110 Train accuracy: 0.622257 Test_accuracy: 0.611285
120 Train accuracy: 0.523511 Test_accuracy: 0.521944
130 Train accuracy: 0.570141 Test_accuracy: 0.568965
140 Train accuracy: 0.534875 Test_accuracy: 0.525078
150 Train accuracy: 0.593652 Test_accuracy: 0.590909
160 Train accuracy: 0.538009 Test_accuracy: 0.

1530 Train accuracy: 0.809561 Test_accuracy: 0.761755
1540 Train accuracy: 0.783699 Test_accuracy: 0.72884
1550 Train accuracy: 0.81348 Test_accuracy: 0.763323
1560 Train accuracy: 0.817006 Test_accuracy: 0.778997
1570 Train accuracy: 0.746082 Test_accuracy: 0.703762
1580 Train accuracy: 0.799373 Test_accuracy: 0.744514
1590 Train accuracy: 0.748433 Test_accuracy: 0.725705
1600 Train accuracy: 0.753527 Test_accuracy: 0.717868
1610 Train accuracy: 0.777429 Test_accuracy: 0.730408
1620 Train accuracy: 0.826019 Test_accuracy: 0.766458
1630 Train accuracy: 0.824451 Test_accuracy: 0.766458
1640 Train accuracy: 0.802508 Test_accuracy: 0.757053
1650 Train accuracy: 0.81779 Test_accuracy: 0.755486
1660 Train accuracy: 0.827586 Test_accuracy: 0.774295
1670 Train accuracy: 0.821708 Test_accuracy: 0.786834
1680 Train accuracy: 0.709639 Test_accuracy: 0.697492
1690 Train accuracy: 0.747649 Test_accuracy: 0.702194
1700 Train accuracy: 0.799373 Test_accuracy: 0.760188
1710 Train accuracy: 0.793103 T

50 Train accuracy: 0.588558 Test_accuracy: 0.597179
60 Train accuracy: 0.630094 Test_accuracy: 0.611285
70 Train accuracy: 0.633229 Test_accuracy: 0.617555
80 Train accuracy: 0.611677 Test_accuracy: 0.600313
90 Train accuracy: 0.572492 Test_accuracy: 0.576802
100 Train accuracy: 0.601881 Test_accuracy: 0.601881
110 Train accuracy: 0.619906 Test_accuracy: 0.637931
120 Train accuracy: 0.649687 Test_accuracy: 0.642633
130 Train accuracy: 0.626176 Test_accuracy: 0.61442
140 Train accuracy: 0.64185 Test_accuracy: 0.65047
150 Train accuracy: 0.644201 Test_accuracy: 0.631661
160 Train accuracy: 0.653605 Test_accuracy: 0.655172
170 Train accuracy: 0.634013 Test_accuracy: 0.615987
180 Train accuracy: 0.664577 Test_accuracy: 0.667712
190 Train accuracy: 0.669279 Test_accuracy: 0.666144
200 Train accuracy: 0.668103 Test_accuracy: 0.670846
210 Train accuracy: 0.612069 Test_accuracy: 0.598746
220 Train accuracy: 0.612461 Test_accuracy: 0.631661
230 Train accuracy: 0.628918 Test_accuracy: 0.628527
2

1590 Train accuracy: 0.982759 Test_accuracy: 0.967085
1600 Train accuracy: 0.981975 Test_accuracy: 0.968652
1610 Train accuracy: 0.980408 Test_accuracy: 0.960815
1620 Train accuracy: 0.979624 Test_accuracy: 0.95768
1630 Train accuracy: 0.975313 Test_accuracy: 0.956113
1640 Train accuracy: 0.978448 Test_accuracy: 0.95768
1650 Train accuracy: 0.979232 Test_accuracy: 0.95768
1660 Train accuracy: 0.974138 Test_accuracy: 0.949843
1670 Train accuracy: 0.965125 Test_accuracy: 0.938872
1680 Train accuracy: 0.977273 Test_accuracy: 0.954545
1690 Train accuracy: 0.967085 Test_accuracy: 0.938872
1700 Train accuracy: 0.963166 Test_accuracy: 0.940439
1710 Train accuracy: 0.976881 Test_accuracy: 0.954545
1720 Train accuracy: 0.979232 Test_accuracy: 0.959248
1730 Train accuracy: 0.979624 Test_accuracy: 0.954545
1740 Train accuracy: 0.978448 Test_accuracy: 0.949843
1750 Train accuracy: 0.973354 Test_accuracy: 0.937304
1760 Train accuracy: 0.958464 Test_accuracy: 0.926332
1770 Train accuracy: 0.982367 T

110 Train accuracy: 0.642241 Test_accuracy: 0.628527
120 Train accuracy: 0.644592 Test_accuracy: 0.65047
130 Train accuracy: 0.661834 Test_accuracy: 0.639498
140 Train accuracy: 0.690831 Test_accuracy: 0.667712
150 Train accuracy: 0.702978 Test_accuracy: 0.691223
160 Train accuracy: 0.716301 Test_accuracy: 0.684953
170 Train accuracy: 0.75 Test_accuracy: 0.72884
180 Train accuracy: 0.679467 Test_accuracy: 0.669279
190 Train accuracy: 0.787226 Test_accuracy: 0.750784
200 Train accuracy: 0.768809 Test_accuracy: 0.753918
210 Train accuracy: 0.826019 Test_accuracy: 0.796238
220 Train accuracy: 0.769984 Test_accuracy: 0.736677
230 Train accuracy: 0.833856 Test_accuracy: 0.791536
240 Train accuracy: 0.83268 Test_accuracy: 0.782132
250 Train accuracy: 0.86442 Test_accuracy: 0.804075
260 Train accuracy: 0.873433 Test_accuracy: 0.821317
270 Train accuracy: 0.89185 Test_accuracy: 0.841693
280 Train accuracy: 0.850313 Test_accuracy: 0.811912
290 Train accuracy: 0.881661 Test_accuracy: 0.824451
30

1660 Train accuracy: 0.982759 Test_accuracy: 0.954545
1670 Train accuracy: 0.974138 Test_accuracy: 0.951411
1680 Train accuracy: 0.948276 Test_accuracy: 0.926332
1690 Train accuracy: 0.976881 Test_accuracy: 0.952978
1700 Train accuracy: 0.982367 Test_accuracy: 0.962382
1710 Train accuracy: 0.981975 Test_accuracy: 0.959248
1720 Train accuracy: 0.979232 Test_accuracy: 0.959248
1730 Train accuracy: 0.983542 Test_accuracy: 0.954545
1740 Train accuracy: 0.97453 Test_accuracy: 0.952978
1750 Train accuracy: 0.978056 Test_accuracy: 0.95768
1760 Train accuracy: 0.97884 Test_accuracy: 0.948276
1770 Train accuracy: 0.981583 Test_accuracy: 0.959248
1780 Train accuracy: 0.976881 Test_accuracy: 0.954545
1790 Train accuracy: 0.981583 Test_accuracy: 0.951411
1800 Train accuracy: 0.97884 Test_accuracy: 0.954545
1810 Train accuracy: 0.98315 Test_accuracy: 0.95768
1820 Train accuracy: 0.985893 Test_accuracy: 0.959248
1830 Train accuracy: 0.98511 Test_accuracy: 0.960815
1840 Train accuracy: 0.980408 Test_

180 Train accuracy: 0.685737 Test_accuracy: 0.65047
190 Train accuracy: 0.682994 Test_accuracy: 0.672414
200 Train accuracy: 0.683386 Test_accuracy: 0.658307
210 Train accuracy: 0.712382 Test_accuracy: 0.688088
220 Train accuracy: 0.738636 Test_accuracy: 0.703762
230 Train accuracy: 0.824843 Test_accuracy: 0.789969
240 Train accuracy: 0.84953 Test_accuracy: 0.815047
250 Train accuracy: 0.875784 Test_accuracy: 0.840125
260 Train accuracy: 0.797414 Test_accuracy: 0.749216
270 Train accuracy: 0.793495 Test_accuracy: 0.763323
280 Train accuracy: 0.739028 Test_accuracy: 0.717868
290 Train accuracy: 0.840909 Test_accuracy: 0.838558
300 Train accuracy: 0.771944 Test_accuracy: 0.736677
310 Train accuracy: 0.870298 Test_accuracy: 0.847962
320 Train accuracy: 0.872649 Test_accuracy: 0.846395
330 Train accuracy: 0.872257 Test_accuracy: 0.84953
340 Train accuracy: 0.79232 Test_accuracy: 0.778997
350 Train accuracy: 0.909091 Test_accuracy: 0.904389
360 Train accuracy: 0.925549 Test_accuracy: 0.9059

1720 Train accuracy: 0.980799 Test_accuracy: 0.940439
1730 Train accuracy: 0.981583 Test_accuracy: 0.932602
1740 Train accuracy: 0.991771 Test_accuracy: 0.95768
1750 Train accuracy: 0.971787 Test_accuracy: 0.923198
1760 Train accuracy: 0.987069 Test_accuracy: 0.952978
1770 Train accuracy: 0.990204 Test_accuracy: 0.948276
1780 Train accuracy: 0.991771 Test_accuracy: 0.956113
1790 Train accuracy: 0.937696 Test_accuracy: 0.9279
1800 Train accuracy: 0.967476 Test_accuracy: 0.929467
1810 Train accuracy: 0.981191 Test_accuracy: 0.954545
1820 Train accuracy: 0.978448 Test_accuracy: 0.946708
1830 Train accuracy: 0.992555 Test_accuracy: 0.956113
1840 Train accuracy: 0.970219 Test_accuracy: 0.931035
1850 Train accuracy: 0.983542 Test_accuracy: 0.931035
1860 Train accuracy: 0.984326 Test_accuracy: 0.95768
1870 Train accuracy: 0.98511 Test_accuracy: 0.940439
1880 Train accuracy: 0.987461 Test_accuracy: 0.954545
1890 Train accuracy: 0.980016 Test_accuracy: 0.942006
1900 Train accuracy: 0.989028 Tes

SystemExit: 

C:\Users\qli45\AppData\Local\Continuum\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
